# COVID-19 cases

## Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import geopandas as gpd
import json
import requests

from pathlib import Path
from zipfile import ZipFile

In [ ]:
from bokeh.io import output_file, output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, GeoJSONDataSource
from bokeh.models import HoverTool, LinearColorMapper, ColorBar
from bokeh.palettes import brewer, Viridis

## Shapefiles

In [ ]:
shapefile = 'gemeente_2020_v1.shp'

In [ ]:
# Download data
download_dir = Path('../downloads')
download_dir.mkdir(exist_ok=True)

url = 'https://www.cbs.nl/-/media/cbs/dossiers/nederland-regionaal/wijk-en-buurtstatistieken/wijkbuurtkaart_2020_v1.zip'
filename = url.split('/')[-1]

if not Path(download_dir /filename).is_file():
    try:
        r = requests.get(url)
        with open(download_dir /filename,'wb') as output_file:
            output_file.write(r.content)
    except:
        pass

In [ ]:
# Extract data
shapefiles_dir = Path('../shapefiles')
shapefiles_dir.mkdir(exist_ok=True)

if not Path(shapefiles_dir /shapefile).is_file():
    with ZipFile(download_dir /filename, 'r') as zipObj:
        zipObj.extractall(shapefiles_dir)

In [ ]:
# Read data
gdf = gpd.read_file(shapefiles_dir / shapefile)
gdf.sample(n=3)

In [ ]:
# Filter rows (i.e. land only) and select columns
gemeente = gdf.loc[gdf.H2O=='NEE', ['GM_NAAM', 'AANT_INW', 'geometry']]
gemeente.sample(n=3)

## COVID-19 data

In [ ]:
base_url = 'https://data.rivm.nl/covid-19'
file_name = 'COVID-19_aantallen_gemeente_per_dag.csv'

In [ ]:
data_dir = Path('../data')
data_dir.mkdir(exist_ok=True)

try:
    rivm = pd.read_csv(f'{base_url}/{file_name}', sep=';')
    print(f'Data downloaded from {base_url}')
except:
    print('Data read from disk')
    rivm = pd.read_csv(data_dir / 'rivm.csv', sep=';')

In [ ]:
datum = '2021-01-05'

In [ ]:
df = rivm.loc[rivm.Date_of_publication == datum, ['Municipality_name', 'Total_reported']]\
.dropna().groupby(by = 'Municipality_name').sum('Total_reported').reset_index()
df.sample(n=5)

## Merge datasets

In [ ]:
gdf = pd.merge(gemeente, df, how='left', left_on='GM_NAAM', right_on='Municipality_name')
gdf.drop('Municipality_name', axis=1, inplace=True)
gdf['AANT_COVID_100K'] = 100_000 / gdf.AANT_INW * gdf.Total_reported # Aantal besmettingen per 100.000 inwoners
gdf.sample(n=5)

## Bokeh

In [ ]:
# Convert to GeoJSON
json_gdf = json.loads(gdf.to_json())

# Convert to string-like object 
json_data = json.dumps(json_gdf)

# GeoJSONDataSource
source = GeoJSONDataSource(geojson = json_data)

In [ ]:
# Set output to notebook
output_notebook()

In [ ]:
palette = brewer['Blues'][256]
palette = palette[::-1] # Reverse color order so that dark red represents high numbers
color_mapper = LinearColorMapper(palette = palette, low = 0, high = gdf['AANT_COVID_100K'].max())
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=18, border_line_color=None, location = (0, 0))

hover = HoverTool(tooltips = [('Gemeente','@GM_NAAM'),
                              ('Aantal besmettingen', '@AANT_COVID_100K{1.1}')])

p = figure(title = 'COVID-10 besmettingen per 100.000 inwoners',
           plot_width=700, plot_height = 800, toolbar_location = None)
p.patches('xs', 'ys', source = source,
          fill_color = {'field' : 'AANT_COVID_100K', 'transform' : color_mapper},
          fill_alpha =  1,
          line_color = 'black',
          line_width = 0.25)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.axis.visible = False
p.add_layout(color_bar, 'right')
p.add_tools(hover)
    
show(p)

In [ ]:
palette = Viridis[256]
color_mapper = LinearColorMapper(palette = palette, low = 0, high = gdf['AANT_COVID_100K'].max())
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=18, border_line_color=None, location = (0, 0))

hover = HoverTool(tooltips = [('Gemeente','@GM_NAAM'),
                              ('Aantal besmettingen', '@AANT_COVID_100K{1.1}')])

p = figure(title = 'COVID-10 besmettingen per 100.000 inwoners',
           plot_width=700, plot_height = 800, toolbar_location = None)
p.patches('xs', 'ys', source = source,
          fill_color = {'field' : 'AANT_COVID_100K', 'transform' : color_mapper},
          fill_alpha =  1,
          line_color = 'black',
          line_width = 0.25)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.axis.visible = False
p.add_layout(color_bar, 'right')
p.add_tools(hover)
    
show(p)